sc.addPyFile("graphframes-0.7.0-spark2.4-s_2.11.jar")

In [ ]:
import os 
os.sys.executable
#sc.addPyFile("graphframes-0.7.0-spark2.4-s_2.11.jar")

In [119]:
from graphframes import *
from pyspark.sql.functions import *

In [120]:

#g.find("(a)-[]->(b); (b)-[]->(d); (e)-[]->(f); (f)-[]->(a)").filter(" a.name = 'Alice'").select("d.name",'e.name').show()
#g.find("(a)-[e]->(b)").filter("b.name = 'Charlie' AND e.relationship = 'follow' ").select("a.name").show()
#g.vertices.join(g.inDegrees, 'id', 'left_outer').filter('inDegree >= 2').select("name").show()
from graphframes import *

from pyspark.sql.functions import *



# Vertics DataFrame

v = spark.createDataFrame([

  ("a", "Alice", 34),

  ("b", "Bob", 36),

  ("c", "Charlie", 37),

  ("d", "David", 29),

  ("e", "Esther", 32),

  ("f", "Fanny", 38),

  ("g", "Gabby", 60)

], ["id", "name", "age"])



# Edges DataFrame

e = spark.createDataFrame([

  ("a", "b", "friend"),

  ("b", "c", "follow"),

  ("c", "b", "follow"),

  ("f", "c", "follow"),

  ("e", "f", "follow"),

  ("e", "d", "friend"),

  ("d", "a", "friend"),

  ("a", "e", "friend"),

  ("g", "e", "follow")

], ["src", "dst", "relationship"])


# Create a GraphFrame

g = GraphFrame(v, e)


#g.vertices.show()

#g.edges.show()


g.find("(a)-[]->(b); (b)-[]->(d); (e)-[]->(f); (f)-[]->(a)").filter(" a.name = 'Alice'").select("d.name",'e.name').show()
g.find("(a)-[]->(b); (b)-[]->(d); !(d)-[]->(a);(e)-[]->(f); (f)-[]->(a);!(e)-[]->(a) ").filter(" a.name = 'Alice'").select("d.name",'e.name').show()
#g.find("(a)-[e]->(b)").filter("b.name = 'Charlie' AND e.relationship = 'follow' ").select("a.name").show()
#g.vertices.join(g.inDegrees, 'id', 'left_outer').filter('inDegree >= 2').select("name").show()

+-------+------+
|   name|  name|
+-------+------+
|  Fanny|Esther|
|  David|Esther|
|Charlie|Esther|
+-------+------+

+-------+------+
|   name|  name|
+-------+------+
|  Fanny|Esther|
|Charlie|Esther|
+-------+------+



In [ ]:
# g.vertices and g.edges are just DataFrames
# You can use any DataFrame API on them

g.edges.filter("src = 'a'").show()

In [ ]:
g.edges.filter("src = 'a'").count()

In [ ]:
# Count the number of followers of c.
# This queries the edge DataFrame.
print g.edges.filter("relationship = 'follow' and dst = 'c'").show()

In [ ]:
# A GraphFrame has additional attributes

g.outDegrees.show()

In [ ]:
g.inDegrees.show()

In [ ]:
g.inDegrees.explain()

In [ ]:
myInDegrees = g.edges.groupBy('dst').count()\
               .withColumnRenamed('dst', 'id').withColumnRenamed('count', 'inDegree')
myInDegrees.show()

In [ ]:
myInDegrees.explain()

In [ ]:
print g.inDegrees.storageLevel

In [ ]:
g.inDegrees.cache()

In [ ]:
print g.inDegrees.storageLevel

In [ ]:
print g.vertices.storageLevel

In [ ]:
g.cache()

In [ ]:
print g.vertices.storageLevel
print g.edges.storageLevel

In [ ]:
# A triplet view of the graph

g.triplets.show()

### Motif Finding

In [122]:
# Search for pairs of vertices with edges in both directions between them.
motifs = g.find("(a)-[]->(b); (b)-[]->(a)").filter('a.id < b.id')
motifs.show()

+------------+----------------+
|           a|               b|
+------------+----------------+
|[b, Bob, 36]|[c, Charlie, 37]|
+------------+----------------+



In [24]:
# Find triangles

triangles = g.find("(a)-[]->(b); (b)-[]->(c); (c)-[]->(a)")
triangles = triangles.filter("a.id < b.id AND a.id < c.id")
triangles.show()

+----------------+----------------+----------------+
|               a|               b|               c|
+----------------+----------------+----------------+
|  [a, Alice, 34]|  [g, Gabby, 60]|  [d, David, 29]|
|  [a, Alice, 34]|[c, Charlie, 37]|  [d, David, 29]|
|  [a, Alice, 34]|  [g, Gabby, 60]|[c, Charlie, 37]|
|  [a, Alice, 34]|    [b, Bob, 36]|[c, Charlie, 37]|
|  [a, Alice, 34]|[c, Charlie, 37]|  [g, Gabby, 60]|
|[c, Charlie, 37]|  [d, David, 29]|  [g, Gabby, 60]|
+----------------+----------------+----------------+



In [15]:
e=sc.parallelize([(1,2),(1,3),(1,4),(2,3),(3,1)]).toDF(["src","dst"])

e1=e.withColumnRenamed('src', 'x').withColumnRenamed('dst','y')
e2=e.withColumnRenamed('src', 'y1').withColumnRenamed('dst','z')
e3=e.withColumnRenamed('src', 'z1').withColumnRenamed('dst','x1')

In [ ]:
triangles.explain()

In [20]:
# e1.show()
# e2.show()
# e3.show()
e1.join(e2,'e1.y=e2.y1','left_outer')


AnalysisException: u'USING column `e1.y=e2.y1` cannot be resolved on the left side of the join. The left-side columns: [x, y];'

In [29]:
# Negation
oneway = g.find("(a)-[]->(b); !(b)-[]->(a)")
oneway.select("a.name",'b').show()

+-------+----------------+
|   name|               b|
+-------+----------------+
| Esther|  [d, David, 29]|
|  Alice|    [b, Bob, 36]|
|Charlie|  [d, David, 29]|
|  Fanny|  [d, David, 29]|
| Esther|  [a, Alice, 34]|
|  Fanny|[c, Charlie, 37]|
|  Fanny|    [b, Bob, 36]|
|  David|  [a, Alice, 34]|
+-------+----------------+



In [ ]:
# Find vertices without incoming edges. This is wrong:
g.find('!()-[]->(a)').show()
# Because negation is implemented as a subtraction

In [ ]:
# Still doesn't work:
g.vertices.join(g.inDegrees, 'id').filter('inDegree=0').show()

In [ ]:
# Why? Because inDegree is computed by a groupBy followed by a count
g.inDegrees.show()

In [ ]:
# Correct way:
g.vertices.join(g.inDegrees, 'id', 'left_outer').filter('inDegree >= 2').show()

In [126]:
g.vertices.select('id').show()
g.inDegrees.select('id').show()

+---+
| id|
+---+
|  a|
|  b|
|  c|
|  d|
|  e|
|  f|
|  g|
+---+

+---+
| id|
+---+
|  f|
|  e|
|  d|
|  c|
|  b|
|  a|
+---+



In [128]:
# Or use subtract:
g.vertices.select('id').subtract(g.inDegrees.select('id')).join(g.vertices,'id').show()

+---+-----+---+
| id| name|age|
+---+-----+---+
|  g|Gabby| 60|
+---+-----+---+



In [ ]:
# More meaningful queries can be expressed by applying filters.
# Question: where is this filter applied?

g.find("(a)-[]->(b); (b)-[]->(a)").filter("b.age > 36").show()

In [ ]:
g.find("(a)-[]->(b); (b)-[]->(a)").filter("b.age > 36").explain()

In [ ]:
# Find chains of 4 vertices such that at least 2 of the 3 edges are "friend" relationships.
# The when function is simvilar to the CASE WHEN in SQL

chain4 = g.find("(a)-[e1]->(b); (b)-[e2]->(c); (c)-[e3]->(d)").where('a!=d')

friendTo1 = lambda e: when(e['relationship'] == 'friend', 1).otherwise(0)

chain4.select('*',friendTo1(chain4['e1']).alias('f1'), \
                  friendTo1(chain4['e2']).alias('f2'), \
                  friendTo1(chain4['e3']).alias('f3')) \
      .where('f1 + f2 + f3 >= 2').select('a', 'b', 'c', 'd').show()

### Subgraphs

In [ ]:
# Select subgraph of users older than 30, and edges of type "friend"
v2 = g.vertices.filter("age > 30")
e2 = g.edges.filter("relationship = 'friend'")
g2 = GraphFrame(v2, e2)
g2.vertices.show()
g2.edges.show()

# GraphFrames does not check if a vertex is isolated (which is OK)
# or if an edge connects two existing vertices (which could cause bugs)


In [ ]:
g2.inDegrees.show()

In [ ]:
# Only keeping edges that connect existing vertices
e3 = e2.join(v2, e2['src'] == v2['id'], 'left_semi') \
       .join(v2, e2['dst'] == v2['id'], 'left_semi') 
g3 = GraphFrame(v2, e3)

In [ ]:
g3.edges.show()

In [ ]:
# Select subgraph based on edges of type "follow"
# pointing from an older user to an younger user.
e4 = g.find("(a)-[e]->(b)")\
        .filter("e.relationship = 'follow'")\
        .filter("a.age > b.age") \
        .select("e.*")
e4.show()

# Only keeping vertices that appear in the edges
v4 = g.vertices.join(e4, g.vertices['id'] == e4['src'], 'leftsemi') \
      .union(g.vertices.join(e4, g.vertices['id'] == e4['dst'], 'leftsemi')) \
      .distinct()
    
# Construct the subgraph
g4 = GraphFrame(v4, e4)
g4.vertices.show()

In [ ]:
g4.triplets.show()

### BFS

In [ ]:
# Starting vertex is 'a'
layers = [g.vertices.select('id').where("id = 'a'")]
visited =  layers[0]

while layers[-1].count() > 0:
    # From the current layer, get all the one-hop neighbors
    d1 = layers[-1].join(g.edges, layers[-1]['id'] == g.edges['src'])
    # Rename the column as 'id', and remove visited verices and duplicates
    d2 = d1.select(d1['dst'].alias('id')) \
           .subtract(visited).distinct()
    layers += [d2]
    visited = visited.union(layers[-1])

In [ ]:
layers[0].show()

In [ ]:
layers[1].show()

In [ ]:
layers[2].show()

In [ ]:
layers[3].show()

In [ ]:
# GraphFrames provides own BFS:

paths = g.bfs("id = 'a'", "age > 36")
paths.show()

### List Ranking

In [ ]:
# -1 denotes end of list
data = [(0, 5), (1, 0), (3, 4), (4, 6), (5, -1), (6,1)]
e = spark.createDataFrame(data, ['src', 'dst'])
v = e.select(col('src').alias('id'), when(e.dst == -1, 0).otherwise(1).alias('d'))
v1 = spark.createDataFrame([(-1, 0)], ['id', 'd'])
v = v.union(v1)
v.show()
e.show()

In [ ]:
while e.filter('dst != -1').count() > 0:
    g = GraphFrame(v, e)
    g.cache()
    v = g.triplets.select(col('src.id').alias('id'), 
                          (col('src.d') + col('dst.d')).alias('d')) \
         .union(v1)
    e = g.find('(a)-[]->(b); (b)-[]->(c)') \
         .select(col('a.id').alias('src'), col('c.id').alias('dst')) \
         .union(e.filter('dst = -1'))
v.show()

In [1]:
from graphframes import *

[u'/tmp/spark-774f52b2-e773-4dad-97bd-8115b75ace1f/userFiles-1e0f9cb8-6545-428b-bdc6-eb51cbf80d95', '', '/usr/spark-2.4.4-bin-hadoop2.7/python/lib/py4j-0.10.7-src.zip', '/usr/spark-2.4.4-bin-hadoop2.7/python', '/home/chris/Downloads/5003', '/home/chris/anaconda3/envs/ipykernel_py2/lib/python27.zip', '/home/chris/anaconda3/envs/ipykernel_py2/lib/python2.7', '/home/chris/anaconda3/envs/ipykernel_py2/lib/python2.7/plat-linux2', '/home/chris/anaconda3/envs/ipykernel_py2/lib/python2.7/lib-tk', '/home/chris/anaconda3/envs/ipykernel_py2/lib/python2.7/lib-old', '/home/chris/anaconda3/envs/ipykernel_py2/lib/python2.7/lib-dynload', '/home/chris/anaconda3/envs/ipykernel_py2/lib/python2.7/site-packages', '/home/chris/anaconda3/envs/ipykernel_py2/lib/python2.7/site-packages/IPython/extensions', '/home/chris/.ipython']


In [1]:
from graphframes import *
# Example Input:
# Vertices DataFrame
v = spark.createDataFrame([
 ("a", "Alice", 34),
 ("b", "Bob", 36),
 ("c", "Charlie", 37)
], ["id", "name", "age"])

# Edges DataFrame
e = spark.createDataFrame([
 ("a", "b", "follow"),
 ("c", "a", "friend"),
 ("b", "c", "follow"),
 ("d", "a", "follow"),
 ("f", "c", "follow"),
 ("f", "d", "follow"),
 ("f", "b", "follow"),
 ("c", "d", "follow"),
 ("g", "a", "friend"),
 ("g", "d", "friend"),
 ("g", "c", "friend"),
 ("e", "a", "follow"),
 ("e", "d", "follow")
], ["src", "dst", "relationship"])
# Create a GraphFrame
g = GraphFrame(v, e)

In [151]:
a='a'
c=e.filter(e.relationship =="follow")
e.filter(c.src==a).select('dst')
#v.join(e,'id').show()
v.filter(v.id==e)
#e.filter(" e.src= 'a' and relationship = 'follow' ").show()

AttributeError: 'DataFrame' object has no attribute '_get_object_id'

In [2]:
e1 = g.edges.filter("relationship = 'follow'")
v1 = e1.groupBy('dst').count().select('dst')
v2 = e1.groupBy('src').count().orderBy('count', ascending=False)
v3 = v2.select('src').subtract(v1.select('dst'))
v4 = v3.join(v2,'src').first()
e.filter(e['src']==v4.src).select('dst').show()

+---+
|dst|
+---+
|  c|
|  d|
|  b|
+---+



In [4]:
g.vertices.show()
g.edges.show()

+---+-------+---+
| id|   name|age|
+---+-------+---+
|  a|  Alice| 34|
|  b|    Bob| 36|
|  c|Charlie| 37|
+---+-------+---+

+---+---+------------+
|src|dst|relationship|
+---+---+------------+
|  a|  b|      follow|
|  a|  c|      follow|
|  b|  c|      friend|
+---+---+------------+



In [118]:
#g.outDegrees.select("id").where('outDegree=2').show()

b=g.edges.filter("relationship = 'follow'")
#help(a['id'])
a= b.groupBy('dst').count().select('dst')
c= b.groupBy('src').count().orderBy('count',ascending=False)
d=c.select('src').subtract(a)
a.show()
c.show()
d.show()

+---+
|dst|
+---+
|  c|
|  b|
+---+

+---+-----+
|src|count|
+---+-----+
|  a|    2|
+---+-----+

+---+
|src|
+---+
|  a|
+---+



In [45]:
a=g.outDegrees.groupby().max('outDegree').collect()[0].asDict()['max(outDegree)']

b=g.outDegrees.filter("outDegree=a").select("id")
#b.show()
#g.edges.filter('src=b').show()

#g.vertices.join(b,'id','left_outer')..show()

2

In [34]:
from graphframes import *
v = spark.createDataFrame([
 ("a", "Alice", 34),
 ("b", "Bob", 36),
 ("c", "Charlie", 37),
 ("d", "David", 29),
 ("e", "Esther", 32),
 ("f", "Fanny", 38),
 ("g", "Gabby", 60)
], ["id", "name", "age"])

# Edges DataFrame
e = spark.createDataFrame([
 ("a", "b", "follow"),
 ("a", "c", "friend"),
 ("a", "d", "friend"),
 ("b", "c", "friend"),
 ("b", "d", "friend"),
 ("c", "a", "friend"),
 ("c", "b", "friend"),
 ("d", "a", "friend"),
 ("d", "b", "friend")
], ["src", "dst", "relationship"])
# Create a GraphFrame
g = GraphFrame(v, e)

In [35]:
g.find('(a)-[e]->(b);(a)-[h]->(d);(d)-[k]->(b)').filter("e.relationship='friend'").filter("h.relationship='friend'").filter("k.relationship='follow'").select('b.name','d.name').distinct().show()
#g.find('(a)-[e]->(b);(b)-[h]->(a)').filter("e.relationship='friend'").filter("h.relationship='friend'").select('a').show()

+----+-----+
|name| name|
+----+-----+
| Bob|Alice|
| Bob|Alice|
+----+-----+



In [21]:
g.edges.filter("relationship = 'friend'").groupby('dst').count().show()
g.edges.filter("relationship = 'friend'").groupby('src').count().show()

+---+-----+
|dst|count|
+---+-----+
|  g|    3|
|  d|    1|
|  c|    3|
|  b|    1|
|  a|    2|
+---+-----+

+---+-----+
|src|count|
+---+-----+
|  g|    3|
|  d|    1|
|  c|    3|
|  b|    1|
|  a|    2|
+---+-----+

